In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from mpl_toolkits import mplot3d

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks, Sequential, Input, Model
from tensorflow.keras.layers import Activation, Dropout, Dense, Add, BatchNormalization, Conv2D, SeparableConv2D, Conv2DTranspose, UpSampling2D, MaxPooling2D, Flatten, concatenate
from sklearn.cluster import KMeans

import math
import scipy
import pickle
import gc 

from IPython.display import display

In [ ]:
tf.random.set_seed(0)


Band 0 - Coastal aerosol

Band 1 - Blue

Band 2 - Green

Band 3 - Red

Band 4 - Near Infrared (NIR)

Band 5 - SWIR 1

Band 6 - SWIR 2

Band 7 - Cirrus

Band 8 - Thermal Infrared (TIRS) 1

Band 9 - Thermal Infrared (TIRS) 2

(Panchromatic Band is Excluded)

In [ ]:
# Train and Validation data from Ecuador/Galapagos
fire_path = ""
nonfire_path = ""
mask_path = ""

fires = pickle.load(open(fire_path, "rb"))
nonfires = pickle.load(open(nonfire_path, "rb"))
masks = pickle.load(open(mask_path, "rb"))
masks = masks.astype("float32")

In [ ]:
# Test data from Guyana/Suriname
test_fire_path = ""
test_mask_path = ""

fires_test = pickle.load(open(test_fire_path, "rb"))
masks_test = pickle.load(open(test_mask_path, "rb"))
masks_test = masks_test.astype("float32")

In [ ]:
def return_idxs(data):
    mins = np.min(data.reshape(data.shape[0], data.shape[1] * data.shape[2] * data.shape[3]), axis=0)
    return np.array(np.nonzero(mins)).tolist()

In [ ]:
idxs_fire = return_idxs(fires)
fires = fires[idxs_fire]
masks = masks[idxs_fire]

idxs_nonfire = return_idxs(nonfires)
nonfires = nonfires[indices_to_keep]

In [ ]:
X_t, X_v, y_t, y_v = train_test_split(fires, masks, test_size=0.15, random_state=0)

In [ ]:
# Min-Max Normalization
X_t_mins = np.min(np.min(np.min(X_t, axis=0), axis=0), axis=0) 
X_t_maxs = np.max(np.max(np.max(X_t, axis=0), axis=0), axis=0) 

In [ ]:
X_t = (X_t - X_t_mins) / (X_t_maxs - X_t_mins)
X_v = (X_v - X_t_mins) / (X_t_maxs - X_t_mins)

In [ ]:
# Normalizing Test Data
fires_test = (fires_test - X_t_mins) / (X_t_maxs - X_t_mins)

In [ ]:
fires_m = fires.shape[0]
nonfires_m = nonfires.shape[0]
tot = fires_m + nonfires_m
print("Number of fires:\t" + str(fires_m))
print("Number of nonfires:\t" + str(nonfires_m))
print("Total:\t" + str(tot))
print("Percentage that are fire:\t" + str(fires_m/tot))
print("Percentage that are nonfire:\t" + str(nonfires_m/tot))

In [ ]:
# Getting a sense of the number of fires in each image
num_clusters = []

for i in range(500):
    fires = np.array(list(np.where(masks[i] == 1))) # 2 x m array where m is the number of pixels that equal 1
    cost = np.inf
    clusters = 0
    cost_threshold = np.sum(masks[i].reshape(masks[i].shape[0], masks[i].shape[1]**2), axis=1) / 3
    while cost * 10 > cost_threshold:
        clusters += 1
        kmeans = KMeans(clusters)
        kmeans.fit(fires.T)
        cost = kmeans.inertia_
    num_clusters.append(clusters)
print(num_clusters)

In [ ]:
# Number of pixels with active fires
# Used for loss function weighting 
num_fires = np.sum(np.sum(masks, axis=-1), axis=-1)
plt.hist(num_fires,bins=50)
plt.yscale('log')
plt.show()

In [ ]:
# Normalize single 128 x 128 x 3 image for plotting
def plot_single(arr):
    epsilon = 1e-4
    reshaped_arr = arr.reshape(arr.shape[0] * arr.shape[1], arr.shape[2])
    arr_min = np.min(reshaped_arr, axis=0)
    arr_max = np.max(reshaped_arr, axis=0)
    reshaped_arr_norm = (reshaped_arr - arr_min) / (arr_max - arr_min + epsilon) 
    img = reshaped_arr_norm.reshape(arr.shape[0], arr.shape[1], arr.shape[2])
    plt.imshow(img)
    plt.show()

In [ ]:
def build_unet_model():
    inputs = Input(shape=(128, 128, 3))
    conv1 = Conv2D(32, (3, 3), padding='same')(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = Activation("relu")(conv1)
    conv1 = Conv2D(32, (3, 3), padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = Activation("relu")(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = Conv2D(64, (3, 3), padding='same')(pool1)
    conv2 = BatchNormalization()(conv2)
    conv2 = Activation("relu")(conv2)
    conv2 = Conv2D(64, (3, 3), padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = Activation("relu")(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), padding='same')(pool2)
    conv3 = BatchNormalization()(conv3)
    conv3 = Activation("relu")(conv3)
    conv3 = Conv2D(128, (3, 3), padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Activation("relu")(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    conv4 = Conv2D(256, (3, 3), padding='same')(pool3)
    conv4 = BatchNormalization()(conv4)
    conv4 = Activation("relu")(conv4)
    conv4 = Conv2D(256, (3, 3), padding='same')(conv4)
    conv4 = BatchNormalization()(conv4)
    conv4 = Activation("relu")(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    up5 = concatenate([Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same')(conv4), conv3], axis=3)
    conv5 = Conv2D(128, (3, 3), padding='same')(up5)
    conv5 = BatchNormalization()(conv5)
    conv5 = Activation("relu")(conv5)
    conv5 = Conv2D(128, (3, 3), padding='same')(conv5)
    conv5 = BatchNormalization()(conv5)
    conv5 = Activation("relu")(conv5)

    up6 = concatenate([Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same')(conv5), conv2], axis=3)
    conv6 = Conv2D(64, (3, 3), padding='same')(up6)
    conv6 = BatchNormalization()(conv6)
    conv6 = Activation("relu")(conv6)
    conv6 = Conv2D(64, (3, 3), padding='same')(conv6)
    conv6 = BatchNormalization()(conv6)
    conv6 = Activation("relu")(conv6)

    up7 = concatenate([Conv2DTranspose(32, (3, 3), strides=(2, 2), padding='same')(conv6), conv1], axis=3)
    conv7 = Conv2D(32, (3, 3), padding='same')(up7)
    conv7 = BatchNormalization()(conv7)
    conv7 = Activation("relu")(conv7)
    conv7 = Conv2D(16, (3, 3), padding='same')(conv7)
    conv7 = BatchNormalization()(conv7)
    conv7 = Activation("relu")(conv7)
  
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(conv7)

    model = Model(inputs=[inputs], outputs=[outputs])

    return model

In [ ]:
model = build_unet_model()
model.summary()

In [ ]:
# tf.keras.utils.plot_model(model, to_file='unet_model.png')

In [ ]:
# Weighting for class imbalance
weight_pos = (masks.shape[0]*masks.shape[1]*masks.shape[2])/np.sum(num_fires) - 1 # weight_pos = 2095.5433356044036

def loss_fn(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    epsilon = 1e-4
    error = - (weight_pos * y_true * tf.math.log(y_pred + epsilon) + (1 - y_true) * tf.math.log(1 - y_pred + epsilon))
    # loss = weight1 * tf.cast(y_true == 1, dtype=tf.float32) * error + weight0 * tf.cast(y_true == 0, dtype=tf.float32) * error
    loss = tf.math.reduce_mean(error, axis=-1)
    
    return loss

In [ ]:
def f2_score(y_true, y_pred):
    y_pred = tf.cast(tf.math.greater_equal(y_pred, 0.5), tf.float32)
    y_true = Flatten()(y_true)
    y_pred = Flatten()(y_pred)
    epsilon = 1e-4
    tp = tf.math.logical_and((y_true == 1), (y_pred == 1))
    tp = tf.cast(tp, tf.float32)
    tp = tf.math.reduce_sum(tp, axis=1)
    fp = tf.math.logical_and((y_true == 0), (y_pred == 1))
    fp = tf.cast(fp, tf.float32)
    fp = tf.math.reduce_sum(fp, axis=1)
    fn = tf.math.logical_and((y_true == 1), (y_pred == 0))
    fn = tf.cast(fn, tf.float32)
    fn = tf.math.reduce_sum(fn, axis=1)
    
    return tp / (tp + 0.2 * fp + 0.8 * fn + epsilon)

In [ ]:
def f1_score(y_true, y_pred):
    y_pred = tf.cast(tf.math.greater_equal(y_pred, 0.5), tf.float32)
    y_true = Flatten()(y_true)
    y_pred = Flatten()(y_pred)
    epsilon = 1e-4
    tp = tf.math.logical_and((y_true == 1), (y_pred == 1))
    tp = tf.cast(tp, tf.float32)
    tp = tf.math.reduce_sum(tp, axis=1)
    fp = tf.math.logical_and((y_true == 0), (y_pred == 1))
    fp = tf.cast(fp, tf.float32)
    fp = tf.math.reduce_sum(fp, axis=1)
    fn = tf.math.logical_and((y_true == 1), (y_pred == 0))
    fn = tf.cast(fn, tf.float32)
    fn = tf.math.reduce_sum(fn, axis=1)
    
    return 2 * tp / (2 * tp + fp + fn + epsilon)

In [ ]:
def evaluate(y_true, y_pred):
    y_pred = tf.cast(tf.math.greater_equal(y_pred, 0.5), tf.float32)
    y_true = Flatten()(y_true)
    y_pred = Flatten()(y_pred)
    epsilon = 1e-4
    tp = tf.math.logical_and((y_true == 1), (y_pred == 1))
    tp = tf.cast(tp, tf.float32)
    tp = tf.math.reduce_sum(tp)
    fp = tf.math.logical_and((y_true == 0), (y_pred == 1))
    fp = tf.cast(fp, tf.float32)
    fp = tf.math.reduce_sum(fp)
    fn = tf.math.logical_and((y_true == 1), (y_pred == 0))
    fn = tf.cast(fn, tf.float32)
    fn = tf.math.reduce_sum(fn)
    tn = tf.math.logical_and((y_true == 0), (y_pred == 0))
    tn = tf.cast(tn, tf.float32)
    tn = tf.math.reduce_sum(tn)
    
    return tp, fp, fn, tn

In [ ]:
alpha0 = 5e-4
def lr_exp_decay(epoch, lr):
    k = 0.1
    return alpha0 * math.exp(-k * epoch)

alpha_schedule = tf.keras.callbacks.LearningRateScheduler(lr_exp_decay)

adam = tf.keras.optimizers.Adam(learning_rate=alpha_schedule)
model.compile(optimizer="adam", loss=loss_fn, metrics=[f2_score, f1_score])

In [ ]:
history = model.fit(X_t, y_t, validation_data=(X_v, y_v), batch_size=32, epochs=100, callbacks=[alpha_schedule])

plt.plot(history.history["loss"])
# Rolling average
# val_loss = history.history["val_loss"][:9] + list(pd.Series(history.history["val_loss"]).rolling(10).mean().dropna())
plt.plot(history.history["val_loss"])
# plt.ylim(0,0.05)
plt.show()

plt.plot(history.history["f2_score"])
# Rolling average
# val_f2 = history.history["val_f2_score"][:9] + list(pd.Series(history.history["val_f2_score"]).rolling(10).mean().dropna())
plt.plot(history.history["val_f2_score"])
plt.ylim(0,1)
plt.show()

plt.plot(history.history["f1_score"])
# Rolling average
# val_f1 = history.history["val_f1_score"][:9] + list(pd.Series(history.history["val_f1_score"]).rolling(10).mean().dropna())
plt.plot(history.history["val_f1_score"])
plt.ylim(0,1)
plt.show()

In [ ]:
# model.save_weights('unet_model_weights.h5')

In [ ]:
# Evaluate performance on validation set
y_pred = model.predict(X_v, batch_size=32)
y_pred = (y_pred >= 0.5).astype("int")
evaluate(y_v, y_pred)

In [ ]:
model.load_weights("/kaggle/input/ecuador-and-galapagos-wildfire-detection/my_model_weights.h5")

In [ ]:
# Evaluate performance on test set (Guyana/Suriname images)
evaluate(masks_test, model.predict(fires_test, batch_size=64))

In [ ]:
shuffler = np.random.permutation(2000)
nonfires_x_pred = nonfires[:,:,:,[2,5,6]]
nonfires_x_pred = nonfires_x_pred[shuffler]

In [ ]:
# Evaluate performance on 2000 nonfire images (ideally should predict every pixel = 0)
evaluate(np.zeros((2000,128,128)), model.predict(nonfires_x_pred, batch_size=32))